In [4]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score

In [16]:
class NaiveBayes():
    def fit(self,X,y):
        self.x = X
        self.y = y
        self.classes = np.unique(y)
        self.parameters = {}
        for i,c in enumerate(self.classes):
            #计算每个种类的平均值，方差，先验概率
            X_index_c = X[np.where(y==c)]   
            X_index_c_mean = np.mean(X_index_c,axis=0,keepdims=True)
            X_index_c_var = np.var(X_index_c,axis=0,keepdims=True)
            parameters = {"mean":X_index_c_mean,"var":X_index_c_var,"prior":X_index_c.shape[0]/X.shape[0]}
            self.parameters['class'+str(c)] = parameters
    def _pdf(self,X,classes):
        #一维高斯分布的概率密度函数
        #贝叶斯估计
        eps = 1e-4
        mean = self.parameters["class"+str(classes)]["mean"]
        var = self.parameters["class"+str((classes))]["var"]
        
        #取对数防止数值溢出
        #enumerate.shape = [m_sample,feature]
        numerator = np.exp(-(X-mean)**2/(2*var+eps))
        denominator = np.sqrt(2*np.pi*var+eps)
        
        #朴素贝叶斯假设（每个特征之间相互独立）
        #P(x1,x2,x3|Y) = P(x1,|Y)*P(x2,|Y)*P(x3,|Y)
        #result.shape = [m_sample,1]
        result = np.sum(np.log(numerator/denominator),axis=1,keepdims=True)
        return result.T
    def _predict(self,X):
        #计算每个分类的概率P(x1,x2,x3) = P(Y)*P(x1|Y)*P(x2|Y)*P(x3|Y)
        output = []
        for y in range(self.classes.shape[0]):
            prior = np.log(self.parameters["class"+str(y)]["prior"])
            posterior = self._pdf(X,y)
            prediction = prior + posterior
            output.append(prediction)
        return output
    def predict(self,X):
        #取概率最大的类别返回预测值
        output = self._predict(X)
        output = np.reshape(output,(self.classes.shape[0],X.shape[0]))
        prediction = np.argmax(output,axis=0)
        return prediction

In [17]:
def main():
    data = datasets.load_digits()
    X = normalize(data.data)
    y = data.target
    
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4)
    print("X_train",X_train.shape)
    clf = NaiveBayes()
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    print("Accuracy:",accuracy)
    
    # Reduce dimension to two using PCA and plot the results
    #Plot().plot_in_2d(X_test,y_pred,title="Naive Bayes",accuracy=accuracy,legend_labels = data.target_names)
    
if __name__ == "__main__":
    main()

X_train (1078, 64)
Accuracy: 0.8984700973574409


C:\Users\wangy\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log
